In [33]:
import os
import time
import random
import numpy as np

import torch
import torch.nn as nn

from torch.autograd import Variable
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [4]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(CharRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        # Layers
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        encoded = self.encoder(x)
        y, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        y = self.decoder(y.view(batch_size, -1))
        return y, hidden

    def forward2(self, x, hidden):
        encoded = self.encoder(x.view(1, -1))
        y, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        y = self.decoder(y.view(1, -1))
        return y, hidden

In [43]:
vocab_size = 825
hidden_size = 32
chunk_size = 10

In [44]:
rnn = CharRNN(vocab_size, hidden_size, vocab_size, n_layers=1)

In [45]:
file = torch.Tensor(np.load('./data/Fallujah - Cerebral Hybridization - 0.npy'))

In [46]:
file_len = len(file)

In [40]:
def get_batch(filename, chunk_len, batch_size):
    
    file = torch.Tensor(np.load(filename))
    file_len = len(file)
    
    input = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    
    for i in range(batch_size):
        
        start = random.randint(0, file_len - chunk_len)
        end = start_index + chunk_len + 1
        
        chunk = file[start:end]
        
        input[i] = chunk[:-1]
        target[i] = chunk[1:]
        
    input = Variable(input)
    target = Variable(target)
    
    return input, target

In [41]:
random_training_set(10, 32)

(tensor([[295, 283, 302, 283, 300, 283, 300, 302, 283, 300],
         [301, 300, 296, 295, 296, 295, 289, 295, 289, 296],
         [181, 181, 181, 181, 181, 178, 178, 178, 178, 178],
         [295, 288, 295, 303, 295, 288, 295, 302, 295, 288],
         [303, 284, 302, 284, 302, 303, 284, 302, 284, 302],
         [303, 295, 288, 295, 302, 295, 288, 295, 296, 295],
         [308, 307, 301, 300, 301, 300, 296, 295, 294, 294],
         [449, 454, 461, 460, 494, 461, 454, 449, 606, 295],
         [329, 278, 277, 275, 112, 118, 118, 118, 118, 118],
         [295, 303, 295, 288, 295, 293, 300, 293, 300, 307],
         [283, 290, 283, 283, 283, 283, 285, 286, 290, 291],
         [283, 329, 118, 118, 118, 118, 118, 118, 118, 118],
         [302, 284, 303, 284, 284, 284, 284, 284, 284, 284],
         [293, 307, 300, 293, 300, 293, 307, 300, 293, 300],
         [281, 281, 281, 281, 281, 281, 281, 673, 460, 453],
         [179, 179, 179, 179, 174, 174, 174, 174, 174, 174],
         [118, 118, 118,